In [ ]:
import numpy as np
import pandas as pd
np.set_printoptions(linewidth=102)
from math import ceil
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral10
from bokeh.models import Legend
from bokeh.layouts import gridplot
from bokeh.models import HoverTool, CustomJS
from bokeh.layouts import row, widgetbox, column, layout
from bokeh.models.widgets import CheckboxGroup

github doesn't support dynamic JavaScript so click this link to view plots:

http://nbviewer.jupyter.org/github/FelicianoAnthony/hbnl_data_viz/blob/master/linked_bokeh.ipynb

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
file = '/vol01/active_projects/anthony/fhd_p4_eeg_pheno_v4_beta.csv'
arr = np.loadtxt(open(file, 'rb'),delimiter=",")
######################
freqs = ['Low theta', 'High theta', 'Low alpha', 'High alpha',
        'Low beta', 'Mid beta', 'High beta']

chans = ['FP1-F3', 'FP2-F4', 'FP1-F7', 'FP2-F8', 'F7-F3',
         'F8-F4', 'F7-T7', 'F8-T8', 'F3-C3', 'F4-C4', 'FZ-CZ',
         'CZ-PZ', 'T7-C3', 'T8-C4', 'T7-P7', 'T8-P8', 'C3-P3', 'C4-P4',
         'P7-P3', 'P8-P4', 'P7-O1', 'P8-O2', 'P3-O1', 'P4-O2', 'PZ-O1',
         'PZ-O2', 'O1-O2', 'CZ-C3', 'CZ-C4', 'PZ-P3', 'PZ-P4', 'F7-C3',
         'F8-C4', 'FP1-FP2', 'F3-FZ', 'FZ-F4']

# set up ages on x axis at intervals of .2 from 12-31 
xaxis = []
count = 11.8
for i in range(96):
    count += float(.2)
    xaxis.append(round(count,2))
    
xaxis_arr = np.array(xaxis)


na = arr.reshape(7,36,6,96)

In [4]:
len(na), len(na[0]), len(na[0][0]), len(na[0][0][0])

(7, 36, 6, 96)

In [5]:
#output_file('test2.html')

figlist = [figure(title= chans[i],plot_width=200,plot_height=200) for i in range(36)]

linelist=[]

count = 0
for fig in figlist:
    
    fig.x_range = figlist[0].x_range
    fig.y_range = figlist[0].y_range
    
    fig.yaxis.axis_label = 'amplitude'
    fig.xaxis.axis_label = 'age'

    line0 = fig.line(xaxis_arr, na[0][count][2], line_width=2, color='#1f77b4')
    line1 = fig.line(xaxis_arr, na[1][count][2], line_width=2, color='#ff7f0e')
    line2 = fig.line(xaxis_arr, na[2][count][2], line_width=2, color='#2ca02c')
    line3 = fig.line(xaxis_arr, na[3][count][2], line_width=2, color='#d62728')
    line4 = fig.line(xaxis_arr, na[4][count][2], line_width=2, color='#9467bd')
    line5 = fig.line(xaxis_arr, na[5][count][2], line_width=2, color='#8c564b')
    line6 = fig.line(xaxis_arr, na[6][count][2], line_width=2, color='#e377c2')
    count+=1
    
    linelist+=[[line0,line1,line2,line3,line4,line5,line6]]
    

#checkbox = CheckboxGroup(labels=['line0','line1','line2', 'line3', 'line4','line5', 'line6'],active=range(7))

checkbox = CheckboxGroup(labels=["low theta", "high theta", "low alpha", "high alpha", "low beta", "mid beta", "high beta"], 
                             active=[0,1,2,3,4,5,6])

iterable = [elem for part in [[('_'.join(['line',str(figid),str(lineid)]),line) for lineid,line in enumerate(elem)] for figid,elem in enumerate(linelist)] for elem in part]

checkbox_code = ''.join([elem[0]+'.visible=checkbox.active.includes('+elem[0].split('_')[-1]+');' for elem in iterable])

checkbox.callback = CustomJS(args={key:value for key,value in iterable+[('checkbox',checkbox)]}, code=checkbox_code)

#grid = gridplot([figlist[:8]+[checkbox],figlist[8:]])

grid = gridplot(figlist, ncols=6)

l = layout([checkbox], [grid])

show(l)

In [6]:
p = figure(plot_width=600, plot_height=600)
a = p.line(xaxis_arr, na[0][0][2], line_width=2, legend='low theta', color='#1f77b4')
b = p.line(xaxis_arr, na[1][0][2], line_width=2, legend='high theta', color='#ff7f0e')
c = p.line(xaxis_arr, na[2][0][2], line_width=2, legend='low alpha', color='#2ca02c')
d = p.line(xaxis_arr, na[3][0][2], line_width=2, legend='high alpha', color='#d62728')

e = p.line(xaxis_arr, na[4][0][2], line_width=2, legend='low beta', color='#9467bd')
f = p.line(xaxis_arr, na[5][0][2], line_width=2, legend='mid beta', color='#8c564b')
g = p.line(xaxis_arr, na[6][0][2], line_width=2, legend='high beta', color='#e377c2')


checkbox = CheckboxGroup(labels=["low theta", "high theta", "low alpha", "high alpha", "low beta", "mid beta", "high beta"], 
                         active=[0,1,2,3,4,5,6])

checkbox.callback = CustomJS(args=dict(line0=a, line1=b, line2=c, line3=d, line4=e, line5=f, line6=g), code="""
    console.log(cb_obj.active);
    line0.visible = false;
    line1.visible = false;
    line2.visible = false;
    line3.visible = false;
    line4.visible = false;
    line5.visible = false;
    line6.visible = false;
    for (i in cb_obj.active) {
        //console.log(cb_obj.active[i]);
        if (cb_obj.active[i] == 0) {
            line0.visible = true;
        } else if (cb_obj.active[i] == 1) {
            line1.visible = true;
        } else if (cb_obj.active[i] == 2) {
            line2.visible = true;
        } else if (cb_obj.active[i] == 3) {
            line3.visible = true;
        } else if (cb_obj.active[i] == 4) {
            line4.visible = true;
        } else if (cb_obj.active[i] == 5) {
            line5.visible = true;
        } else if (cb_obj.active[i] == 6) {
            line6.visible = true;
        }
    }
""")

layout = row(p, widgetbox(checkbox))

show(layout)